# ANLP 2019 - Assignment 3


*Rodrigo Lopez Portillo Alcocer, 805606* (enter your name/student id number here)

<div class="alert alert-block alert-danger">Due: Wednesday, December 4th (before class)</div>

<div class="alert alert-block alert-info">
**NOTE**<br><br>

Please first fill in your name and id number at the top of the assignment, and **rename** the assignment file to **yourlastname-anlp-3.ipynb**<br><br>
Problems and questions are given in blue boxes like this one. All grey and white boxes must be filled by you (they either require code or a (brief!) discussion). <br><br>
Please hand in your assignment by the deadline via Moodle. In case of questions, you can contact the TAs or David via the usual channels.
</div>

<div class="alert alert-block alert-info">
In this assignment, you will implement a bigram part-of-speech (POS) tagger based on Hidden Markov Models from scratch. Using NLTK is disallowed, except for the modules explicitly listed below. For this, you will need to develop and utilize the following modules:<br>
1. Corpus reader and writer<br>
2. Training procedure, including smoothing<br>
3. Viterbi tagging, including unknown word handling <br>
4. Evaluation<br>
The task is mostly very straightforward, but each step requires careful design. Thus, we suggest you proceed in the following way.
</div>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import nltk

## Problem 1: Viterbi Algorithm [33 points]

<div class="alert alert-block alert-info">
First, implement the Viterbi algorithm for finding the optimal state (tag) sequence given the sequence of observations (words). <br><br>
In order to test your implementation, verify that you compute the correct state sequence for some examples from Eisner's ice cream model (see lecture) for which the solutions are known.<br><br>
Demonstrate that your algorithm computes the correct state sequence for ['3','1','3'] as in the lecture.<br><br>
Make sure that your algorithm is correct before proceeding to the other tasks! In order to do this, please also test your module with a longer observation sequence. 
</div>

In [6]:
#EISNER_STATES = ['C','H']
#EISNER_INITIAL_PROBS = {'C': 0.2, 'H': 0.8}
#EISNER_TRANSITIONS = {'C': {'C':0.6, 'H': 0.4}, 'H': {'C':0.3, 'H':0.7}}
#EISNER_EMISSIONS = {'C': {'1':0.5,'2':0.4,'3':0.1},'H': {'1':0.2, '2':0.4,'3':0.4}}


# your code goes here
states1 = ['C','H']
pi1 = {'C': 0.2, 'H': 0.8}
a1 = {'C': {'C':0.6, 'H': 0.4}, 'H': {'C':0.3, 'H':0.7}}
b1 = {'C': {'1':0.5,'2':0.4,'3':0.1},'H': {'1':0.2, '2':0.4,'3':0.4}}
unique_words = ['1', '2', '3'] #to handle unk in Eisner case

def viterbi(obs, states, pi, a, b):
    #obs list of observations
    #states list of possible states
    #pi dict with initial probs
    #a dict with transition probs
    #b dict with emission probs
    
    global unique_words #to check if the current observation is unk
    #faster to search for it in unique_words that in whole corpus
    np.random.seed(42)
    obs = [i.lower() for i in obs]
    T = len(states)
    N = len(obs)
    vit = np.zeros([T, N])  #init both matrices
    backp = np.ones([T, N])
    #base case from initial prob and first observation
    for i in range(T):
        #adding how to handle unk 
        if obs[0] in unique_words:
            vit[i,0] = np.exp(np.log(pi[states[i]]) + np.log(b[states[i]][obs[0]]))
        else:#unk handling
            #random choice of state
            rand_state = np.random.choice(states)
            vit[i,0] = pi[rand_state]
        backp[i,0] = 0
    
    #recursive step
    for j in range(1, N):
        for i in range(T):
            if obs[j] in unique_words:
                temp = [np.exp(np.log(vit[s, j-1]) + np.log(a[states[s]][states[i]]) + np.log(b[states[i]][obs[j]])) for s in range(T)]
                vit[i, j] = np.max(temp)
            else: #unk handling
                temp = [vit[s, j-1] for s in range(T)]
                vit[i, j] = np.max(temp)                            
            backp[i, j] = int(np.argmax(temp))
    
    #termination steps
    best_path_prob = np.max(vit[:,-1])
    best_path_pointer = np.argmax(vit[:,-1])
    
    #finally, retrieving the best_prob path
    path = np.zeros([N]).astype(int)
    for i in range(N):
        if i == 0:
            path[i] = best_path_pointer
        else:
            path[i] = backp[path[i-1],N-i]
    #reversing it to have the correct order
    path = path[::-1]
    
    #from indices [1,0,1,1,1,0] to states [H,C,H,H,H,C] 
    #
    path = [states[path[i]] for i in range(N)]
    return path, best_path_prob

In [7]:
obs1 = np.random.randint(1,4,20).astype(str)
viterbi(obs1, states1, pi1, a1, b1);

obs2 = ['3','1', '3']
viterbi(obs2, states1, pi1, a1, b1)

(['H', 'H', 'H'], 0.012543999999999996)

## Problem 2: HMM Training [33 points]

<div class="alert alert-block alert-info">
Second, learn parameters, i.e. the initial, transition, and emission probabilities, for your HMM from *annotated* data. Implement a maximum likelihood training procedure (with smoothing) for supervised learning of HMMs. We will be using the Wall Street Journal corpus (part of the Penn Treebank). It is a licensed corpus, so please do not redistribute the files. The zip archive provided on Moodle contains a training set, a test set, and an evaluation set. The training set (`wsj_tagged_train.tt`) and the evaluation set (`wsj_tagged_eval.tt`) are written in the commonly used CoNLL format. They are text files with two colums; the first column contains the words, the POS tags are in the second column, and empty lines delimit sentences. The test set (`wsj_tagged_test.t`) is a copy of the evaluation set with tags stripped. You should tag this test set using your tagger and then compare your results with the gold-standard ones in the provided tagged evaluation file. The corpus uses the Penn Treebank tagset.<br><br>
You are welcome to use any NLTK data structures from the two modules `nltk.corpus.reader` (and submodules) and `nltk.probability`. The latter includes a number of smoothing procedures, which you may want to apply to your corpus frequency counts. Take care to get NLTK to make the smoothed probability distributions sum to one. Experiment with unsmoothed distributions, Laplace add-one smoothing, and at least one further smoothing procedure.<br><br>
Note that your tagger will initially fail to produce output for sentences that contain words you haven't seen in training. If you have such a word $w$ appear at sentence position $t$, you will have $b_j(w) = 0$ for all states/tags $j$, and therefore $V_t(j) = 0$ for all $j$. Adapt your tagger by implementing the following crude approach to unknown words. Whenever you get $V_t(j) = 0$ for all $j$ because of an unknown word at position $t$, pretend that $b_j(w) = 1$ for all $j$. This will basically set $V_t(j) = max_iV_{t-1}(i)a_{ij}$, and allow you to interpolate the missing POS tag based on the transition probabilities alone.<br><br>
Note 2: In order to avoid additional problems with zero-probability transitions when applying your model to the test set, make sure that you tag the corpus sentence by sentence (i.e., compute the optimal tag sequence for each sentence independently). 
</div>

In [8]:
CORPUS_train = "wsj_tagged/wsj_tagged_train.tt"
CORPUS_gold = "wsj_tagged/wsj_tagged_eval.tt"
CORPUS_test = "wsj_tagged/wsj_tagged_test.t"

In [9]:
corp_train = nltk.corpus.reader.conll.ConllCorpusReader(root = '', fileids=CORPUS_train, columntypes = ['words', 'pos'])
corp_gold = nltk.corpus.reader.conll.ConllCorpusReader(root = '', fileids=CORPUS_gold, columntypes = ['words', 'pos'])
corp_test = nltk.corpus.reader.conll.ConllCorpusReader(root = '', fileids=CORPUS_test, columntypes = ['words', 'pos'])

In [12]:
#getting the initial state probabilities
#we can directly compute the probabilities without having to smooth in this case
#if it has an initial probability then it has at least one element.
fdist = nltk.probability.FreqDist(tagged_tuple[1] for tagged_tuple in corp_train.tagged_words())
total_words = np.sum([fdist[key] for key in fdist.keys()])
for key in fdist.keys():
    fdist[key] /= total_words
#now we have the dict with our initial probabilities
pi = fdist

In [14]:
#now to create the transition probs

def tagged_sentence_condit_counter(read_corpus):
    #counts transitions in the given corpus and returns them in the cfdist form
    cfdist = nltk.probability.ConditionalFreqDist()
    for tagged_sentence in read_corpus.tagged_sents():
        i=0
        while i < len(tagged_sentence)-1: #while we still have at least two words
            current = tagged_sentence[i][1]
            sig = tagged_sentence[i+1][1]
            cfdist[current][sig] +=1  #!!! main step for creating the structure
            i +=1
    return cfdist

cfdist_transition = tagged_sentence_condit_counter(corp_train)

#smoothing with k=1
# we have to keep track of how many counts we add
#number of tags squared
#----------------------------------------------------
#NEVERMIND :P
#this counting is very inefficient. It is tempting to get it counting the number of keys in cfdist_transition
#but if we had one or more tags that only appeared at the end of the sentences they wouldn't
#be represented in the keys of cfdcfdist_transition, but inside one of its sub dicts
#because it never appeared in current, only as sig

#is there a faster way than this?

#counting with np.unique
#added_counts = len(np.unique([corp_train.tagged_words()[i][1] for i in range(len(corp_train.tagged_words()))]))**2

#trying with base.set
#added_counts = len(set([corp_train.tagged_words()[i][1] for i in range(len(corp_train.tagged_words()))]))**2
#--------------------------------------------------------
#finally the version that worked, using the previous fdist
added_counts = len(fdist)**2

#this not only smoothes the entries that already existed, but also adds all the other posibilities to our transition matrix (dict in this case)
for tag1 in cfdist_transition:
    for tag2 in cfdist_transition:
        cfdist_transition[tag1][tag2] += 1

            
#to check if now cfdist contains all possible transitions we can do the following check
#len(np.unique([len(cfdist_transition[key]) for key in cfdist_transition])) == 1
#if TRUE, then all starting points can reach the same number of targets

#normalizing inside each class
for origin in cfdist_transition.keys():
    local_count = sum([cfdist_transition[origin][i] for i in cfdist_transition[origin]])
    for destination in cfdist_transition[origin].keys():
        cfdist_transition[origin][destination] /= local_count

a = cfdist_transition

#checking that they sum ~1
#for i in a:
#    print(sum([a[i][j] for j in a[i]]))

In [8]:
#finally, let's get the emission probabilities
#similar structure than for the transitions
def tagged_words_condit_counter(read_corpus):
    #counts emission probabilities in the given corpus and returns them in the cfdist form
    cfdist = nltk.probability.ConditionalFreqDist()
    unique_words = []
    for tagged_word in read_corpus.tagged_words():
        tag = tagged_word[1]
        word = tagged_word[0].lower()
        unique_words.append(word)
        #adding to a list the lower case words to produce a unique_words list after lower-casing
        cfdist[tag][word] += 1
    return cfdist, set(unique_words)

cfdist_emission, unique_words = tagged_words_condit_counter(corp_train)

#smoothing with k=1
#added counts in this case is size of the vocab * number of tags

for tag in cfdist_emission:
    for word in unique_words:
        cfdist_emission[tag][word] += 1
        
        

#after smoothing all cfdist_emission[key] have the length of the whole vocab
#added_counts_emissions = np.sqrt(added_counts) * len(cfdist_emission['IN'])
            
#to check if now cfdist_emission contains all possible emissions we can do the following check
#len(np.unique([len(cfdist_emission[key]) for key in cfdist_emission])) == 1
#if TRUE, then all states can generate all possible observations

#normalizing
#total_emissions = total_words + added_counts_emissions
for tag in cfdist_emission.keys():
    local_count = sum([cfdist_emission[tag][i] for i in cfdist_emission[tag]])
    for word in cfdist_emission[tag].keys():
        cfdist_emission[tag][word] /= local_count

b = cfdist_emission

#checking that they sum ~1
#for i in a:
#    print(sum([a[i][j] for j in a[i]]))

In [9]:
#getting a list of the states
states = [i for i in cfdist_transition]

In [10]:
obs = ['I', 'am', 'going', 'to', 'Berlin', 'for', 'the', 'weekend', '.']
viterbi(obs, states, pi, a, b )

(['PRP', 'VBP', 'VBG', 'TO', 'VB', 'IN', 'DT', 'NN', '.'],
 7.08826171341732e-26)

In [19]:
#trying with an invented "word"
obs = ['I', 'am', 'kjabsdv', 'to', 'Berlin', 'for', 'the', 'weekend', '.']
viterbi(obs, states, pi, a, b )

(['PRP', 'VBP', 'VBN', 'TO', 'VB', 'IN', 'DT', 'NN', '.'],
 1.5709593674225556e-22)

<div class="alert alert-block alert-info">
Explain which smoothing procedures you used and any observations.
</div>

### NOTES
* For the word counts we set all words to lower case. I am not sure how much that could have affected the tags. Woods last name would now be counted same as woods plural noun. Even with this, I believe more harm would be done if we count as separate nouns their capitalized and lowered versions.

* How to handle if first observation is unk? For now, we choose randomly an initial state. 
* Should't we also set the a_ij to 1 whenever we encounter obs[j] = unk? Same logic as the hint for the emission case, since we don't know to which tag it belongs we don't have transition coef a_ij.

* Didn't have time to implement other smoothing methods.


## Problem 3: Evaluation [34 points]

<div class="alert alert-block alert-info">
Once you have trained a model, evaluate it on the unseen data from the test set. Run the Viterbi algorithm with each of your models, and output a tagged corpus in the two-column CoNLL format (*.tt). Use the provided evaluation script `tagging_eval.py`, which you can run on a gold annotated file and your own tagging results.<br><br>
Run it on the output of your tagger and the evaluation set and report your results. 
</div>

### TO DO
* eval result
* eval with other k-smoothings
* eval with smoothing functions

In [20]:
# too inefficient!
# How to make it better?
def corpus_viterbi(read_corpus):
    global states
    global pi
    global a
    global b
    tagged_sentences = []
    for sentence in read_corpus.sents():
        tagged_sentences.append(viterbi(sentence, states, pi, a, b)[0])
    return tagged_sentences

In [21]:
tagging_test = corpus_viterbi(corp_test)

C:\Users\rodri\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log


In [37]:
def eval_tags(fname_gold, fname_model):
    n = 0
    pr_counts = {}
    with open(fname_gold) as gold, open(fname_model) as model:
        for line_gold, line_model in zip(gold, model):
            line_gold = line_gold.rstrip()
            line_model = line_model.rstrip()
            n += 1
            if len(line_gold) == 0:
                continue

            word_gold, tag_gold = line_gold.split("\t")
            word_model, tag_model = line_model.split("\t")

            # word forms should match in gold and system file
            if word_model != word_gold:
                raise ValueError("\nError in line {}: Words in gold and model files are different. Either your gold "
                                 "file is broken or your model file was generated incorrectly.".format(n))

            if tag_gold not in pr_counts:
                pr_counts[tag_gold] = {"correct": 0, "total_model": 0, "total_gold": 0}
            if tag_model not in pr_counts:
                pr_counts[tag_model] = {"correct": 0, "total_model": 0, "total_gold": 0}
            pr_counts[tag_gold]["total_gold"] += 1
            pr_counts[tag_model]["total_model"] += 1
            if tag_gold == tag_model:
                pr_counts[tag_model]["correct"] += 1

        print("\nComparing gold file {} with model output file {}...".format(fname_gold, fname_model))
        correct = 0
        total = 0
        print("\nTAG / Precision / Recall / F1\n")
        for tag, counts in pr_counts.items():
            correct += counts["correct"]
            total += counts["total_gold"]
            if 0 in counts.values():
                print("Tag {} was never correctly assigned. One of precision, recall or F1 is undefined!".format(tag))
            else:
                precision = counts["correct"] / counts["total_model"]
                recall = counts["correct"] / counts["total_gold"]
                f1 = (2 * precision * recall) / (precision + recall)
                print("{:5s}: {:4f} / {:4f} / {:4f}".format(tag, precision, recall, f1))

        print("\nOverall Accuracy: {:5f}".format(correct / total))


<div class="alert alert-block alert-info">
Discuss the results of the different tagger versions.
</div>

*your discussion goes here*

## Extra Credit [10 points]

<div class="alert alert-block alert-info">
The task is challenging as it stands. However, feel free to go further for extra credit, e.g. by doing one of the following: implement better unknown word handling, use a trigram tagger, plot a learning curve for your tagger (accuracy as a function of training data size), plot a speed vs. sentence length curve.
</div>

In [ ]:
# space for your code

*your discussion*